### sparse Autoencoder for states
#### code adapted from darkefyre

In [399]:
import tensorflow as tf
import keras
import numpy as np
import math
import os
import random
import numpy as np
import pandas as pd
from pandas import DataFrame
import pickle
import math
import copy
from sklearn.model_selection import train_test_split

In [2]:
state_features = ['Albumin', 'Arterial_BE', 'Arterial_lactate', 'Arterial_pH', 'BUN', 'CO2_mEqL', 'Calcium', 'Chloride', 'Creatinine', 'DiaBP', 'FiO2_1', 'GCS', 'Glucose', 'HCO3', 'HR', 'Hb', 'INR', 'Ionised_Ca', 'Magnesium', 'MeanBP', 'PT', 'PTT', 'PaO2_FiO2', 'Platelets_count', 'Potassium', 'RR', 'SGOT', 'SGPT', 'SIRS', 'SOFA', 'Shock_Index', 'Sodium', 'SpO2', 'SysBP', 'Temp_C', 'Total_bili', 'WBC_count', 'Weight_kg', 'age', 'elixhauser', 'gender', 'mechvent', 'output_4hourly', 'output_total', 'paCO2', 'paO2', 're_admission']
print (state_features)

['Albumin', 'Arterial_BE', 'Arterial_lactate', 'Arterial_pH', 'BUN', 'CO2_mEqL', 'Calcium', 'Chloride', 'Creatinine', 'DiaBP', 'FiO2_1', 'GCS', 'Glucose', 'HCO3', 'HR', 'Hb', 'INR', 'Ionised_Ca', 'Magnesium', 'MeanBP', 'PT', 'PTT', 'PaO2_FiO2', 'Platelets_count', 'Potassium', 'RR', 'SGOT', 'SGPT', 'SIRS', 'SOFA', 'Shock_Index', 'Sodium', 'SpO2', 'SysBP', 'Temp_C', 'Total_bili', 'WBC_count', 'Weight_kg', 'age', 'elixhauser', 'gender', 'mechvent', 'output_4hourly', 'output_total', 'paCO2', 'paO2', 're_admission']


In [398]:
train_df = pd.read_csv('../../data/train_scaled.csv')

In [400]:
train_df.head()

,bloc,icustayid,gender,age,elixhauser,re_admission,SOFA,SIRS,Weight_kg,GCS,...,input_4hourly_tev,output_total,output_4hourly,cumulated_balance_tev,sedation,mechvent,rrt,died_in_hosp,mortality_90d,reward
0,0.000000,12.0,1.0,0.203388,0.000000,0.0,0.304348,0.25,0.183842,1.000000,...,0.000000,0.000000,0.000000,0.177479,0.0,0.0,0.0,0.0,0.0,0.0
1,0.222560,12.0,1.0,0.203388,0.000000,0.0,0.130435,0.50,0.183842,1.000000,...,0.000000,0.616278,0.782139,0.175772,0.0,0.0,0.0,0.0,0.0,0.0
2,0.356608,12.0,1.0,0.203388,0.000000,0.0,0.086957,0.50,0.183842,1.000000,...,0.000000,0.659309,0.767260,0.174328,0.0,0.0,0.0,0.0,0.0,0.0
3,0.452837,12.0,1.0,0.203388,0.000000,0.0,0.217391,0.50,0.183842,1.000000,...,0.000000,0.659309,0.000000,0.174328,0.0,0.0,0.0,0.0,0.0,15.0
4,0.000000,14.0,0.0,0.909010,0.142857,0.0,0.217391,0.50,0.204211,0.047619,...,0.822795,0.570801,0.657302,0.179999,1.0,1.0,0.0,0.0,1.0,0.0


In [401]:
keys = train_df.keys()

In [402]:
X_train, X_val, _, _ = train_test_split(train_df, train_df, test_size=0.2, random_state=42)

In [403]:
train_df = pd.DataFrame(X_train, columns=keys)
val_df = pd.DataFrame(X_val, columns=keys)

In [405]:
train_df.sample(2).loc[:,state_features].head()

,Albumin,Arterial_BE,Arterial_lactate,Arterial_pH,BUN,CO2_mEqL,Calcium,Chloride,Creatinine,DiaBP,...,Weight_kg,age,elixhauser,gender,mechvent,output_4hourly,output_total,paCO2,paO2,re_admission
108113,0.437690,0.377737,0.038827,0.492105,0.276891,0.218112,0.450262,0.376316,0.270786,0.615686,...,0.247122,0.694198,0.214286,0.0,0.0,0.695240,0.706632,0.225482,0.079137,0.0
27188,0.468085,0.481752,0.045253,0.726316,0.696384,0.234694,0.481675,0.322039,0.513332,0.555294,...,0.327110,0.861096,0.428571,0.0,1.0,0.814046,0.834580,0.135727,0.142446,1.0


In [406]:
n_input = len(state_features)
n_hidden_1 = 200
weights = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.zeros([n_hidden_1])),
    'decoder_b1': tf.Variable(tf.zeros([n_input])),
}

In [407]:
class AutoEncoder:
    def __init__(self):
        self.input_size = n_input
        self.batch_size = tf.placeholder(tf.int32)
        self.sparsity_dist = tf.fill([self.batch_size, n_hidden_1], 0.05)
        self.X = tf.placeholder("float", [None, self.input_size])
        self.layer_1_a = tf.nn.sigmoid(tf.add(tf.matmul(self.X, weights['encoder_h1']),
                                   biases['encoder_b1']))
        self.layer_1_d = tf.nn.sigmoid(tf.add(tf.matmul(self.layer_1_a, weights['decoder_h1']),
                                   biases['decoder_b1']))
        self.target_y = self.X
        self.kl = tf.reduce_mean(tf.reduce_sum(tf.multiply(self.sparsity_dist, tf.log(self.sparsity_dist/self.layer_1_a)) + \
                                   tf.multiply((1-self.sparsity_dist),tf.log((1-self.sparsity_dist)/(1-self.layer_1_a))), axis =1))
        #self.kl = tf.reduce_mean(tf.contrib.keras.losses.kullback_leibler_divergence(self.sparsity_dist, self.layer_1_a))  
    # regularisation term logic - loss at the start is roughly 60, of this 0.5 is reconstruction loss.  
        self.loss = tf.reduce_mean(tf.pow(self.target_y - self.layer_1_d, 2)) + 0.0001 * self.kl
        #self.loss = tf.reduce_mean(tf.pow(self.target_y - self.layer_1_d, 2))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(self.loss)

In [410]:
def eval_on_val():
    x = val_df.loc[:,state_features]
    l = sess.run([autoencoder.loss], feed_dict={autoencoder.X:x, autoencoder.batch_size:X_val.shape[0]})
    print ("validation loss is: ", l)

In [409]:
batch_size = 100
autoencoder = AutoEncoder()
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for step in range(100000):
        sum_loss = 0
        a = scaled_train_df.sample(n=batch_size)
        cur_state = a.loc[:, state_features]
        _, l = sess.run([autoencoder.optimizer, autoencoder.loss], feed_dict={autoencoder.X:cur_state, autoencoder.batch_size : batch_size})
        sum_loss += l
        if step % 1000==0 and step > 0:
            print ("step is ", step )
            print ("total loss is ", sum_loss)
            sum_loss = 0
        if step % 20000 == 0 and step >0:
            b = scaled_train_df.sample(n=1)
            cur_state_trial = b.loc[:, state_features]
            target_y, reconstruction = sess.run([autoencoder.target_y,autoencoder.layer_1_d],feed_dict={autoencoder.X:cur_state_trial})
            print (reconstruction)
            print (target_y)
        if step % 20000 == 0 and step >0:
            eval_on_val()

step is  1000
total loss is  0.0728789195418
step is  2000
total loss is  0.0533630847931
step is  3000
total loss is  0.0435242801905
step is  4000
total loss is  0.0347269810736
step is  5000
total loss is  0.0282772444189
step is  6000
total loss is  0.0224697198719
step is  7000
total loss is  0.018863145262
step is  8000
total loss is  0.0152579843998
step is  9000
total loss is  0.013226756826
step is  10000
total loss is  0.0117595009506
step is  11000
total loss is  0.00952850002795
step is  12000
total loss is  0.00848221220076
step is  13000
total loss is  0.00725245429203
step is  14000
total loss is  0.00608120206743
step is  15000
total loss is  0.00556242000312
step is  16000
total loss is  0.0052120196633
step is  17000
total loss is  0.00385940121487
step is  18000
total loss is  0.0034402385354
step is  19000
total loss is  0.00307540316135
step is  20000
total loss is  0.00272108078934
[[ 0.6353519   0.47738478  0.05082005  0.65062267  0.09031203  0.24574506
   0.4093

InvalidArgumentError: Incompatible shapes: [30149,200] vs. [30158,200]
	 [[Node: truediv_121 = RealDiv[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](Fill_68, Sigmoid_135)]]

Caused by op 'truediv_121', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  F

In [ ]:
# train_states = scaled_train_df.loc[:,state_features]
# train_repr = sess.run([autoencoder.layer_1_a], feed_dict={autoencoder.X:train_states})

# train_autoencode = pd.DataFrame(train_repr[0])
# train_autoencode['vaso_input'] = scaled_train_df['vaso_input']
# train_autoencode['iv_input'] = scaled_train_df['iv_input']
# train_autoencode['reward'] = train_df['reward']
# train_autoencode['icustayid'] = train_df['icustayid']
# # test_states = df_test.ix[:,state_features]
# test_repr = sess.run([autoencoder.layer_1_a], feed_dict={autoencoder.X:test_states})
# test_autoencode = pd.DataFrame(test_repr[0])
# test_autoencode['vaso_input'] = df_test['vaso_input']
# test_autoencode['iv_input'] = df_test['iv_input']
# test_autoencode['reward'] = df_test['reward']
# test_autoencode['icustayid'] = df_test['icustayid']